In [1]:
!pip install python-chess 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 58.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=f7e896778be400283b42b4bd5255d9f7b76f8af6df7b58c4187e468fc4853528
  Stored in directory: /root/.cache/pip/wheels/83/1f/4e/8f4300f7dd554eb8de70ddfed96e94d3d030ace10c5b53d447
Successfully built chess


In [2]:
!pip install onnxruntime-openvino

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 MB 23.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.9 MB/s eta 0:00:00


In [10]:
import chess
import chess.polyglot
import onnxruntime as ort
import numpy as np
import requests
import json
import threading
import time
import math
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

# ============================================================================
# CONFIGURATION
# ============================================================================
API_TOKEN = ""
PTH_WEIGHTS_PATH = "/kaggle/input/teacher-model-weights/resnet20_student_v3.pth"
BOOK_PATH = "/kaggle/input/openings/Titans.bin"
ONNX_MODEL_PATH = "resnet20_student_v3_int8.onnx"

# 🚀 FINAL TUNED PARAMETERS
C_PUCT = 0.9            
MAX_THINK_TIME = 20.0    # CPU Safety Limit
TIME_FLEX_PERCENT = 0.05

In [4]:
# ==========================================
# 1. MODEL ARCHITECTURE
# ==========================================
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels, channels, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channels)
    def forward(self, x):
        return torch.relu(x + self.bn2(self.conv2(torch.relu(self.bn1(self.conv1(x))))))

class ChessResNet(nn.Module):
    def __init__(self, num_features=18, num_moves=4096, num_res_blocks=20, num_channels=256):
        super().__init__()
        self.conv_input = nn.Conv2d(num_features, num_channels, 3, padding=1, bias=False)
        self.bn_input = nn.BatchNorm2d(num_channels)
        self.res_tower = nn.Sequential(*[ResidualBlock(num_channels) for _ in range(num_res_blocks)])
        self.p_conv = nn.Conv2d(num_channels, 32, 1); self.p_bn = nn.BatchNorm2d(32)
        self.p_fc = nn.Linear(32 * 8 * 8, num_moves) 
        self.v_conv = nn.Conv2d(num_channels, 32, 1); self.v_bn = nn.BatchNorm2d(32)
        self.v_fc1 = nn.Linear(32 * 8 * 8, 128); self.v_fc2 = nn.Linear(128, 3) 

    def forward(self, x):
        x = torch.relu(self.bn_input(self.conv_input(x)))
        x = self.res_tower(x)
        p = self.p_fc(torch.relu(self.p_bn(self.p_conv(x))).view(x.size(0), -1))
        v = self.v_fc2(torch.relu(self.v_fc1(torch.relu(self.v_bn(self.v_conv(x))).view(x.size(0), -1))))
        return p, v

# ==========================================
# 2. CONVERSION UTILITY
# ==========================================
def prepare_model():
    if os.path.exists(ONNX_MODEL_PATH): os.remove(ONNX_MODEL_PATH)
    print(f"🔄 Converting PyTorch weights from {PTH_WEIGHTS_PATH}...")
    
    model = ChessResNet()
    if not os.path.exists(PTH_WEIGHTS_PATH):
        print("❌ Weights file not found!"); return

    try:
        state_dict = torch.load(PTH_WEIGHTS_PATH, map_location='cpu')
        new_state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
        model.load_state_dict(new_state_dict)
    except Exception as e:
        print(f"❌ Load Error: {e}"); return

    model.eval()
    dummy_input = torch.randn(1, 18, 8, 8)
    
    print("🛠️ Exporting to ONNX...")
    torch.onnx.export(model, dummy_input, "temp.onnx", export_params=True, opset_version=11, 
                      do_constant_folding=False, input_names=['input'], output_names=['policy', 'value'],
                      dynamic_axes={'input': {0: 'batch'}, 'policy': {0: 'batch'}, 'value': {0: 'batch'}})
    
    print("📉 Quantizing model...")
    quantize_dynamic("temp.onnx", ONNX_MODEL_PATH, weight_type=QuantType.QUInt8)
    if os.path.exists("temp.onnx"): os.remove("temp.onnx")
    print(f"🎉 Engine Ready: {ONNX_MODEL_PATH}")

if __name__ == "__main__":
    prepare_model()

🔄 Converting PyTorch weights from /kaggle/input/teacher-model-weights/resnet20_student_v3.pth...
🛠️ Exporting to ONNX...


/tmp/ipykernel_55/4115331654.py:54: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(model, dummy_input, "temp.onnx", export_params=True, opset_version=11,


📉 Quantizing model...


🎉 Engine Ready: resnet20_student_v3_int8.onnx


In [5]:
class ONNXEngine:
    def __init__(self, model_path):
        
        # CPU-ONLY Mode
        self.session = ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])
        print(f"✅ Inference Engine Loaded on: {self.session.get_providers()[0]}")
            
        self.input_name = self.session.get_inputs()[0].name
        
    def predict_batch(self, fens):
        batch_tensors = []
        for fen in fens:
            board = chess.Board(fen)
            batch_tensors.append(self._board_to_numpy(board))
            
        if not batch_tensors: return [], []
            
        full_batch = np.concatenate(batch_tensors, axis=0)
        outs = self.session.run(None, {self.input_name: full_batch})
        
        policy_logits_batch = outs[0]
        value_logits_batch = outs[1]
        
        results_policy = []
        results_value = []
        
        for i in range(len(fens)):
            p_logits = policy_logits_batch[i]
            exp_p = np.exp(p_logits - np.max(p_logits))
            p_probs = exp_p / exp_p.sum()
            results_policy.append(p_probs)
            
            v_logits = value_logits_batch[i]
            exp_v = np.exp(v_logits - np.max(v_logits))
            v_probs = exp_v / exp_v.sum() 
            
            # WDL Score: Win + 0.5 * Draw
            score = v_probs[0] + (0.5 * v_probs[1])
            
            if " w " not in fens[i]: 
                score = 1.0 - score
                
            results_value.append(score)
            
        return results_policy, results_value

    def _board_to_numpy(self, board):
        PIECE_MAP = {'P':0, 'N':1, 'B':2, 'R':3, 'Q':4, 'K':5, 'p':6, 'n':7, 'b':8, 'r':9, 'q':10, 'k':11}
        matrix = np.zeros((18, 8, 8), dtype=np.float32)
        for sq, piece in board.piece_map().items():
            row, col = divmod(sq, 8)
            matrix[PIECE_MAP[piece.symbol()], 7 - row, col] = 1.0
        if board.has_kingside_castling_rights(chess.WHITE): matrix[12, :, :] = 1.0
        if board.has_queenside_castling_rights(chess.WHITE): matrix[13, :, :] = 1.0
        if board.has_kingside_castling_rights(chess.BLACK): matrix[14, :, :] = 1.0
        if board.has_queenside_castling_rights(chess.BLACK): matrix[15, :, :] = 1.0
        if board.turn == chess.WHITE: matrix[16, :, :] = 1.0
        matrix[17, :, :] = board.halfmove_clock / 100.0
        return np.expand_dims(matrix, axis=0)

In [6]:
# ==========================================
# 3. BATCH MCTS
# ==========================================
BATCH_SIZE = 8   

class MCTSNode:
    def __init__(self, game_state, parent=None, move=None, prior=0):
        self.game_state = game_state; self.parent = parent; self.move = move 
        self.children = {}; self.prior = prior; self.visit_count = 0; self.value_sum = 0
        self.is_expanded = False
    @property
    def value(self): return self.value_sum / self.visit_count if self.visit_count > 0 else 0

class BatchMCTS:
    def __init__(self, engine, c_puct=0.8):
        self.engine = engine
        self.c_puct = c_puct # Fixed at 0.8
        self.root = None 

    def step_root(self, move):
        if self.root is not None and move in self.root.children:
            self.root = self.root.children[move]
            self.root.parent = None 
        else: self.root = None 

    def run(self, fen, time_limit=5.0):
        if self.root is None or self.root.game_state.fen() != fen:
            self.root = MCTSNode(chess.Board(fen))
            pol, val = self.engine.predict_batch([fen])
            self._expand(self.root, pol[0])
            self.root.value_sum += val[0]; self.root.visit_count += 1

        start_time = time.time(); simulations = 0
        
        while True:
            # Time check
            if simulations % 20 == 0:
                if (time.time() - start_time) > time_limit: break
            
            leaf_nodes = []
            
            # --- SELECTION PHASE ---
            for _ in range(BATCH_SIZE):
                node = self.root
                
                while node.is_expanded and len(node.children) > 0:
                    node = self._select_child(node)
                
                # "Virtual Visit": Temporarily increase visit count so next selection
                # in this batch picks a different node. We DO NOT punish value.
                node.visit_count += 1 
                leaf_nodes.append(node)

            # --- PREPARE INFERENCE ---
            valid_leafs = []; valid_fens = []
            for node in leaf_nodes:
                # Revert "Virtual Visit" before real update
                node.visit_count -= 1 
                
                outcome = node.game_state.outcome()
                if outcome:
                    val = 1.0 if outcome.winner == chess.WHITE else 0.0 if outcome.winner == chess.BLACK else 0.5
                    self._backpropagate(node, val)
                else:
                    valid_leafs.append(node); valid_fens.append(node.game_state.fen())
            
            if not valid_fens: continue
            
            # --- BATCH INFERENCE ---
            policies, values = self.engine.predict_batch(valid_fens)
            
            # --- EXPANSION & BACKPROP ---
            for i, node in enumerate(valid_leafs):
                self._expand(node, policies[i])
                self._backpropagate(node, values[i])
            simulations += BATCH_SIZE

        if not self.root.children: return list(self.root.game_state.legal_moves)[0], "Mate"
        best_child = max(self.root.children.values(), key=lambda n: n.visit_count)
        
        # Stats Display
        wr = best_child.value if self.root.game_state.turn == chess.WHITE else 1.0 - best_child.value
        score_str = f"{(-400 * math.log(1/wr - 1))/100.0:+.2f}" if 0.001 < wr < 0.999 else ("+M" if wr > 0.5 else "-M")
        
        print(f"⚡ Sim: {simulations} | Depth: {self._get_depth(best_child)} | Score: {score_str}")
        return best_child.move, score_str

    def _select_child(self, node):
        best_score = -float('inf'); best_child = None
        for child in node.children.values():
            q = child.value if node.game_state.turn == chess.WHITE else 1.0 - child.value
            u = self.c_puct * child.prior * (math.sqrt(node.visit_count) / (1 + child.visit_count))
            if (q + u) > best_score: best_score = q + u; best_child = child
        return best_child

    def _expand(self, node, policy):
        node.is_expanded = True; sum_probs = 0
        for move in node.game_state.legal_moves:
            idx = (move.from_square * 64) + move.to_square
            if idx < 4096:
                prob = policy[idx]; sum_probs += prob
                next_state = node.game_state.copy(stack=False); next_state.push(move)
                node.children[move] = MCTSNode(next_state, parent=node, move=move, prior=prob)
        if sum_probs > 0:
            for child in node.children.values(): child.prior /= sum_probs

    def _backpropagate(self, node, value):
        while node: 
            node.visit_count += 1
            node.value_sum += value
            node = node.parent

    def _get_depth(self, node):
        d = 0
        while node: 
            d += 1; 
            if not node.children: break
            node = max(node.children.values(), key=lambda n: n.visit_count)
        return d

In [7]:
class LichessClient:
    def __init__(self, token, engine):
        self.token = token; self.engine = engine
        self.base_url = "https://lichess.org/api"; self.headers = {"Authorization": f"Bearer {token}"}
        try:
            me = requests.get(f"{self.base_url}/account", headers=self.headers).json()
            if 'error' in me: print(f"❌ Auth Error: {me['error']}"); self.id = None
            else:
                self.username = me['username']; self.id = me['id']
                print(f"✅ Logged in as: {self.username}")
                requests.post(f"{self.base_url}/bot/account/upgrade", headers=self.headers)
        except: self.id = None

    def get_move(self, fen, mcts_instance, remaining_time):
        board = chess.Board(fen)
        if os.path.exists(BOOK_PATH) and len(board.move_stack) < 20:
            try:
                with chess.polyglot.open_reader(BOOK_PATH) as reader:
                    entry = reader.weighted_choice(board)
                    print(f"📘 Book Move: {entry.move.uci()}")
                    mcts_instance.step_root(entry.move) 
                    return entry.move
            except: pass
        
        think_time = max(0.5, min(remaining_time * TIME_FLEX_PERCENT, MAX_THINK_TIME))
        print(f"🤔 Thinking ({think_time:.1f}s)...")
        move, score = mcts_instance.run(fen, time_limit=think_time)
        mcts_instance.step_root(move)
        print(f"💡 Move: {move.uci()} | Eval: {score}")
        return move

    def stream_game(self, game_id):
        print(f"🏁 Starting Game {game_id}...")
        game_mcts = BatchMCTS(self.engine, c_puct=C_PUCT) # Use global config
        url = f"{self.base_url}/bot/game/stream/{game_id}"
        last_fen = ""
        
        with requests.get(url, headers=self.headers, stream=True) as response:
            for line in response.iter_lines():
                if not line: continue
                try: event = json.loads(line.decode('utf-8'))
                except: continue
                
                if event['type'] == 'gameFull':
                    is_white = (event['white'].get('id', '').lower() == self.id.lower())
                    state = event['state']
                    moves = state['moves']; wtime = state['wtime']; btime = state['btime']
                elif event['type'] == 'gameState':
                    moves = event['moves']; wtime = event['wtime']; btime = event['btime']
                else: continue
                
                board = chess.Board()
                move_list = []
                if moves:
                    for m in moves.split(): 
                        m_uci = chess.Move.from_uci(m)
                        board.push(m_uci); move_list.append(m_uci)
                
                if game_mcts.root and game_mcts.root.game_state.fen() != board.fen():
                    if move_list:
                        try: game_mcts.step_root(move_list[-1])
                        except: game_mcts.root = None 

                if board.is_game_over(): print(f"🏆 Game Over: {game_id}"); return
                
                my_turn = (board.turn == chess.WHITE and is_white) or (board.turn == chess.BLACK and not is_white)
                if my_turn and board.fen() != last_fen:
                    last_fen = board.fen()
                    my_clock = (wtime if is_white else btime) / 1000.0
                    best_move = self.get_move(board.fen(), game_mcts, my_clock)
                    requests.post(f"{self.base_url}/bot/game/{game_id}/move/{best_move.uci()}", headers=self.headers)

    def start_listening(self):
        if not self.id: return
        print("📡 Online & Waiting for Challenges...")
        url = f"{self.base_url}/stream/event"
        while True:
            try:
                with requests.get(url, headers=self.headers, stream=True) as response:
                    for line in response.iter_lines():
                        if not line: continue
                        event = json.loads(line.decode('utf-8'))
                        if event['type'] == 'challenge':
                            print(f"⚔️ Challenge from {event['challenge']['challenger']['name']}!")
                            requests.post(f"{self.base_url}/challenge/{event['challenge']['id']}/accept", headers=self.headers)
                        elif event['type'] == 'gameStart':
                            threading.Thread(target=self.stream_game, args=(event['game']['id'],)).start()
            except Exception as e:
                print(f"⚠️ Disconnected ({e}). Retrying in 5s..."); time.sleep(5)

In [11]:
if __name__ == "__main__":
    engine = ONNXEngine(ONNX_MODEL_PATH)
    bot = LichessClient(API_TOKEN, engine)
    bot.start_listening()

✅ Inference Engine Loaded on: CPUExecutionProvider
✅ Logged in as: Pradybot
📡 Online & Waiting for Challenges...
🏁 Starting Game MGqZREDh...
📘 Book Move: c7c5
📘 Book Move: b8c6
📘 Book Move: c5d4
🤔 Thinking (20.0s)...
⚡ Sim: 720 | Depth: 6 | Score: +0.39
💡 Move: e7e5 | Eval: +0.39
🤔 Thinking (20.0s)...
⚡ Sim: 800 | Depth: 6 | Score: -0.12
💡 Move: d7d6 | Eval: -0.12
🤔 Thinking (20.0s)...
⚡ Sim: 760 | Depth: 9 | Score: -1.15
💡 Move: d6e5 | Eval: -1.15
🤔 Thinking (20.0s)...
⚡ Sim: 800 | Depth: 8 | Score: -1.51
💡 Move: c6d8 | Eval: -1.51
🤔 Thinking (20.0s)...
⚡ Sim: 800 | Depth: 5 | Score: -0.61
💡 Move: d8c6 | Eval: -0.61
🤔 Thinking (20.0s)...
⚡ Sim: 800 | Depth: 8 | Score: -2.01
💡 Move: f8b4 | Eval: -2.01
🤔 Thinking (20.0s)...
⚡ Sim: 760 | Depth: 8 | Score: -0.18
💡 Move: b4a5 | Eval: -0.18
🤔 Thinking (20.0s)...
⚡ Sim: 760 | Depth: 7 | Score: -0.43
💡 Move: e8e7 | Eval: -0.43
🤔 Thinking (20.0s)...
⚡ Sim: 760 | Depth: 6 | Score: -0.80
💡 Move: a5c7 | Eval: -0.80
🤔 Thinking (20.0s)...
⚡ Sim: 80

KeyboardInterrupt: 